<a href="https://colab.research.google.com/github/saerarawas/AAI_635O_B11_202520-Recommender-System/blob/main/Week2/Movie_Recommendation_System_using_BPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# loading movie data
import pandas as pd
import os # import the os module to work with file paths
# loading movie data
import pandas as pd
import os # import the os module to work with file paths
# loading movie data
import pandas as pd
import os # import the os module to work with file paths
# loading movie data
import pandas as pd
import os # import the os module to work with file paths

# Define the file path
item_filepath = '/content/u.item'

# Check if file exists and update path accordingly if necessary
if not os.path.exists(item_filepath):
    # Assuming dataset is in current directory
    item_filepath = 'ml-100k/u.item'
    print(f"Updated item_filepath to: {item_filepath}")

# Continue loading data using updated filepath
item_df = pd.read_csv(item_filepath, sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date',
                            'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                            "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                            'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western'])

item_df = item_df[['movie_id', 'movie_title']]
item_df.head()
if not os.path.exists(item_filepath):
    # Assuming dataset is in current directory
    item_filepath = 'ml-100k/u.item'
    print(f"Updated item_filepath to: {item_filepath}")

# Continue loading data using updated filepath
item_df = pd.read_csv(item_filepath, sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date',
                            'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                            "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                            'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western'])

item_df = item_df[['movie_id', 'movie_title']]
item_df.head()
if not os.path.exists(item_filepath):
    # Assuming dataset is in current directory
    item_filepath = 'ml-100k/u.item'
    print(f"Updated item_filepath to: {item_filepath}")

# Continue loading data using updated filepath
item_df = pd.read_csv(item_filepath, sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date',
                            'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                            "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                            'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western'])

item_df = item_df[['movie_id', 'movie_title']]
item_df.head()
if not os.path.exists(item_filepath):
    # Assuming dataset is in current directory
    item_filepath = 'ml-100k/u.item'
    print(f"Updated item_filepath to: {item_filepath}")

# Continue loading data using updated filepath
item_df = pd.read_csv(item_filepath, sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date',
                            'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                            "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                            'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                            'Thriller', 'War', 'Western'])

item_df = item_df[['movie_id', 'movie_title']]
item_df.head()

,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [11]:
user_item_rating_filepath = '/content/u.data'
df = pd.read_csv(user_item_rating_filepath, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
df = df.drop(['timestamp'], axis=1)
df.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [12]:
# Convert explicit ratings to implicit feedback
df['interaction'] = (df['rating'] >= 4).astype(int)
df.head()

,user_id,item_id,rating,interaction
0,196,242,3,0
1,186,302,3,0
2,22,377,1,0
3,244,51,2,0
4,166,346,1,0


In [13]:
from scipy.sparse import csr_matrix

# Create a sparse matrix for user-item interactions
interaction_matrix = csr_matrix((df['interaction'], (df['user_id'], df['item_id'])))

In [14]:
import numpy as np

def generate_triplets(df, num_negative_samples=1):
    """
    Generate (user, positive_item, negative_item) triplets for BPR training.

    Parameters:
    - df: DataFrame with columns ['user_id', 'item_id', 'interaction'].
    - num_negative_samples: Number of negative samples per positive interaction.

    Returns:
    - triplets: List of (user, positive_item, negative_item) tuples.
    """
    triplets = []
    all_items = set(df['item_id'].unique())  # All unique items in the dataset

    for user_id, group in df[df['interaction'] == 1].groupby('user_id'):
        positive_items = set(group['item_id'].tolist())  # Positive items for this user
        negative_items = list(all_items - positive_items)  # Items not interacted with

        for i in positive_items:
            # Randomly sample negative items for this user
            sampled_negatives = np.random.choice(negative_items, num_negative_samples, replace=False)
            for j in sampled_negatives:
                triplets.append((user_id, i, j))

    return triplets

# Generate triplets
triplets = generate_triplets(df)
print(f"Generated {len(triplets)} triplets")

Generated 55375 triplets


In [15]:
import subprocess
import sys

try:
    # Quietly install lightfm
    subprocess.check_call([sys.executable, "-m", "pip", "install", "lightfm", "--quiet"])
    print("LightFM installed successfully!")
except subprocess.CalledProcessError:
    print("Failed to install LightFM. Please check your environment and try again.")

LightFM installed successfully!


In [17]:
from scipy.sparse import coo_matrix

# Create a sparse matrix for implicit feedback
interaction_matrix = coo_matrix(
    (df['interaction'], (df['user_id'] - 1, df['item_id'] - 1))
)

In [18]:
from lightfm import LightFM

# Initialize the BPR model
model = LightFM(loss='bpr')

# Fit the model
model.fit(interaction_matrix, epochs=30, num_threads=4)

In [19]:
# Predict scores for a user and item
user_id = 87
item_ids = np.arange(interaction_matrix.shape[1])  # All items
scores = model.predict(user_id, item_ids)

# Recommend top 5 items
top_items = np.argsort(-scores)[:5]

# Map item IDs to movie titles using the item_df DataFrame
recommended_movies = item_df[item_df['movie_id'].isin(top_items)]['movie_title'].tolist()

print("Recommended movies for user {}:".format(user_id))
for movie in recommended_movies:
    print(movie)

Recommended movies for user 87:
Last of the Mohicans, The (1992)
Secrets & Lies (1996)
Ice Storm, The (1997)
Rainmaker, The (1997)
Jackal, The (1997)
